In [41]:
import numpy as np
from random import *
import tkinter as tk
import tkinter.messagebox
import time

%matplotlib inline 
from matplotlib import pyplot as plt

In [53]:
class Game_2048: 
    """
    The board game 2048 Supreme.
    
    Attributes:
    -----------
    board: the game board
    m: height of the board
    n: width of the board
    goal: the target goal to win the game
    prime: the base number of the game
    game_over: 1 = game over, 0 = game ongoing
    win: whether or not we won
    actions: allowable actions at each timestep
    history: list of previous board states
    move_hist: list of the distances moved by tiles during previous states
        -most recent move_hist contains distance travelled to current state
    fill_history: list of the coordinates randomly filled by a number
    """
    board = np.zeros(4)
    m = 4
    n = 4
    goal = 2048
    prime = 2
    game_over = 0
    win = 0
    actions = ["left", "up", "right", "down", '0', '1', '2', '3', 
               0, 1, 2, 3, "a", "w", "d", "s"]
    history = []
    move_history = []
    fill_history = []
    merge_history = []
    
    def __init__(self, m = 4, n = 4, prime = 2, goal = 2048, board_array = None):
        """
        Creates a new instance of the game. Sets up the board, prime number, and goal number. 
        
        By default, it creates a standard 4 x 4 board. 
        
        Inputs
        ------
        m = # rows or the height of the board
        n = # columns or the width of the board
        prime = the base prime number of this game
        goal = the goal number to achieve to win
        board_array = a custom np array to use as a board
        """
        if board_array is None:
            self.board = np.zeros((m, n))
            self.m = m
            self.n = n
        else:
            self.board = board_array
            
        self.goal = goal
        self.prime = prime
        self.game_over = 0
        self.win = 0
        self.history = []
        self.move_history = []
        self.fill_history = []
        self.merge_history = []
        
        for i in range(0, self.prime):
            self.board = self.fill_random_cell(self.board, self.prime)
        
    def fill_random_cell(self, board, prime):
        """
        Fills a random empty cell in the board with the prime value.
        
        Inputs
        ------
        board: board you want to fill a cell with
        prime: the prime number to fill in the board
        
        Outputs:
        --------
        modBoard: the modified board with a cell filled in
        """
        modBoard = np.copy(board)
        i, j = (modBoard == 0).nonzero()
        if i.size != 0:
            rnd = randint(0, i.size - 1)
            # i[rnd] is row=y, j[rnd] is col=x
            modBoard[i[rnd], j[rnd]] = prime
            self.fill_history.append((i[rnd], j[rnd]))
        else:
            self.game_over = 1
        return modBoard
            
    def get_successor_2(self, board):
        """
        Moves all blocks to the left in the given board 
        
        Proceeds by column, moving each item in the column as far left as possible individually.
        
        If a block encounters another block, then it stops unless they are the same block. If so,
        then it counts if we can add a number of those blocks together equal to prime. If so, then
        they are combined. 
        
        Then it fills a random cell on the board. Then checks if it is game over. If a cell could not
        be filled, then it is game over. 
        
        Inputs:
        -------
        board: the board you want to enact moveLeft on
        
        Outputs:
        --------
        modBoard: the left shifted board
        move_hist: a dict mapping old coords to new positions on modBoard
        
        """
        m, n = board.shape
        modBoard = np.copy(board)
        merged = np.zeros((m, n))
        # Our movement history. Note it uses inverse notation of (y, x) coords
        move_hist = {}
        
        # i is current column == x
        for i in range(n):
            # j is current row == y
            for j in range(m):
                # current = the tile we are currently shifting
                current = modBoard[j][i]
                # Do nothing to zero tiles
                if current == 0:
                    continue
                    
                # Use y, x to keep track of where we have shifted (j, i) to SO FAR
                y, x = j, i
                
                # Let k be the column to the LEFT of y that we are trying to shift to
                print([i-t for t in range(1, i+1)])
                for k in [i-t for t in range(1, i+1)]:
                    dest_tile = modBoard[j][k]
                    
                    # if left adjacent tile is 0, we can shift to it
                    # since current != 0 due to above if statement
                    if dest_tile == 0:
                        modBoard[j][k] = current
                        modBoard[y][x] = 0
                        y = j
                        x = k
                        # set our movement history to this place
                        move_hist[(j, i)] = (y, x)
                        continue
                        
                    # if left adjacent tile is not the same, or has been merged, 
                    # then we can't do anything
                    elif (dest_tile != current) or merged[j][i] == 1:
                        break
                        
                    # left adjacent tile IS the same and is NOT 0
                    # therefore we must check if a merge is allowed
                    else:
                        # count remembers how many of the same tile we've seen
                        # if we find (self.prime) of the same we can merge them
                        count = 1
                        combine = False
                        # remembers the locations we saw the same tile, 
                        # which we need to convert to 0 if we merge
                        # written in inverse notation (y, x)
                        zeros = [(y, x)]
                        
                        # let c be the column to the LEFT where we are forward checking
                        # for the presence of the same current tile
                        for c in [i-h for h in range(k, i+1)]: 
                            
                            # found another of the same tile
                            if modBoard[j][c] == current:
                                count += 1
                                
                                # we have found the exact number of tiles needed for merge
                                if count == self.prime: 
                                    # merge all the tiles we've found at location (j, c)
                                    modBoard[j][c] = current * self.prime
                                    # mark that we've merged here already
                                    merged[j][c] = 1
                                    
                                    for index in zeros:
                                        # replace marked locations with 0,
                                        # note that the tiles have moved to (j, c) during merge
                                        z_y = index[0]
                                        z_x = index[1]
                                        modBoard[z_y][z_x] = 0
                                        move_hist[(z_y, z_x)] = (j, c)
                                    break
                                    
                                # if we haven't found the required count, then we won't stop here
                                # so mark this location, (j, c), as a potential place to put a zero
                                zeros.append((j, c))
                                    
                            # if we find any non-current tile we must stop searching
                            else:
                                break
        return modBoard
    
    def get_successor(self, board): 
        """
        Moves all blocks to the left in the given board 
        
        Proceeds by column vector, scanning and shifting the column vector until all items have been placed.
        
        If a block encounters another block, then it stops unless they are the same block. If so,
        then it counts if we can add a number of those blocks together equal to prime. If so, then
        they are combined. 
        
        Then it fills a random cell on the board. Then checks if it is game over. If a cell could not
        be filled, then it is game over. 
        
        Inputs:
        -------
        board: the board you want to enact moveLeft on
        
        Outputs:
        --------
        modBoard: the left shifted board
        merge_array: an array indicating where merges occured on the board
        move_hist: an array mapping original tiles to new positions on modBoard
        """
        
        # TODO: FIX THIS
        def merge_vectors(start_index, index_1, index_2, board, merge_array, move_hist):
            """
            Merge two vectors by shifting vector_2 as far left as 
            possible into vector 1.
            
            Returns the new (vector_1, vector_2) after the shift is complete.
            
            Inputs:
            ------
            start_index: the original index of the vector being shifted
            index_1: the index of the first vector on the board
            index_2: the index of the second vector on the board
            board: the board that you are extracting the vectors from
            merge_array: a 2D array with 1 in entries where we are not allowed to merge
            move_dist: an array mapping each tile in the array to the distance it travelled
                        - uses inverse notation (y, x)
            
            Outputs:
            -------
            vec_1: the new vector 1 after merge
            vec_2: the new vector 2 after merge
            merge_array: an updated 2D array with 1 in entries where we are not allowed to merge
            move_dist: an updated array mapping each tile in the array to the distance travelled to
                        - uses inverse notation (y, x)
            """
            
            vec_1 = np.copy(board[:, index_1])
            vec_2 = np.copy(board[:, index_2])
            ill_merge_1 = np.copy(merge_array[:, index_1])
            ill_merge_2 = np.copy(merge_array[:, index_2])
            move_dist_copy = np.copy(move_hist)
            
            for i in range(len(vec_1)):
                # Nothing to shift
                if vec_2[i] == 0:
                    continue
                # Space available to shift into
                elif vec_1[i] == 0 and vec_2[i] != 0:
                    vec_1[i] = vec_2[i]
                    vec_2[i] = 0
                    # moved from index_2 -> index_1
                    # Uses inverse (y, x) notation
                    move_dist_copy[i][start_index] += 1
                    
                # Merge is possible
                # Check not just if equal but if a multiple of a recent prime
                # TODO: RECORD THE LOCATIONS OF THE (ORIGIN MOVED) -> (END MOVED)
                
                elif (vec_1[i] == vec_2[i]) and (ill_merge_1[i] == 0) and (ill_merge_2[i] == 0):
                    vec_1[i] = vec_2[i] + vec_1[i]
                    vec_2[i] = 0
                    ill_merge_1[i] = 1
                    # moved from index_2 -> index_1
                    move_dist_copy[i][start_index] += 1
                    
            merge_array[:, index_1] = ill_merge_1
            merge_array[:, index_2] = ill_merge_2
            return vec_1, vec_2, merge_array, move_dist_copy
        
        rows, columns = board.shape
        mod_board = np.copy(board)
        merge_array = np.zeros((rows, columns))
        # define datatype as a list of 2 64-byte integers [x, y]
        move_dist = np.zeros((rows, columns))
        
        for col_index in range(columns):
            #column_vector = board[:, col_index] 
            # array with 0 if a position has been merged, 1 otherwise
            for shift_index in range(col_index):
                index_1 = col_index-shift_index-1
                index_2 = col_index-shift_index
                vec_1, vec_2, new_merge_array, new_move_dist = merge_vectors(col_index, index_1, index_2, 
                                                                             mod_board, merge_array, move_dist)
                """print("vec_1", vec_1)
                print("vec_1 index", col_index-shift_index-1)
                print("vec_2", vec_2)
                print("vec_2 index", col_index-shift_index)"""
                mod_board[:, index_1] = vec_1
                mod_board[:, index_2] = vec_2
                #print("mod_board\n", mod_board)
                merge_array = new_merge_array
                move_dist = new_move_dist
        return mod_board, move_dist, merge_array
                
    
    def get_action(self, command):
        """
        Takes in a varaible command and returns the corresponding action.
        
        Inputs:
        ------
        command: can be anything, from a number (0, 1, 2, 3) to a digit (left, up, right, down)
        
        Outputs:
        movement: a digit corresponding to the command; (0, 1, 2, 3) only
        """
        if command in self.actions:
            movement = self.actions.index(command) % 4
        else:
            movement = None
        return movement
    
    def move(self, command = 0):
        """
        Moves the board in the desired direction. Moving up, right, or down is just
        moving left on a rotated board. 
        
        Inputs:
        -------
        direction: The direction you wish to move, the following are all equivalent.
            "left" = '0' = 0
            "up" = '1' = 1
            "right" = '2' = 2
            "down" = '3' = 3
        """
        if command in self.actions:
            board = np.copy(self.board)
            movement = self.get_action(command)
            if movement == None:
                print('Error, retrieved incorrect command')
                pass  
            # rotate board to perform left shift movement
            board = np.rot90(board, movement)
            # find desired values from movement operation
            board, move_dist, merge_array = self.get_successor(board)
            # rotate back to acquire correctly oriented version
            board = np.rot90(board, -movement)
            move_dist = np.rot90(move_dist, -movement)
            merge_array = np.rot90(merge_array, -movement)
            
            if np.array_equal(board, self.board) == False:
                board = self.fill_random_cell(board, self.prime)
                # adds current state to history
                self.history.append(self.board)
                self.move_history.append((move_dist, movement))
                self.merge_history.append(merge_array)
                # overrides current state with new one
                self.board = board
        else:
            print("Error. Please enter one of the following: left, up, right, down. \n \
            Alternatively, type: 0, 1, 2, or 3")
    
    def is_game_over(self):
        """
        Checks if the game is over.
        
        Outputs:
        --------
        1: game is over
        0 : game is ongoing
        """
        if self.game_over == 1 or self.is_win() == 1:
            return 1
        else:
            return 0
        
    def is_win(self):
        """
        Checks if we won the game
        
        Outputs:
        --------
        1: we won
        0: we lost
        """
        for block in np.nditer(self.board):
            if block == self.goal:
                return 1
        return 0
    
    def get_val(self, x, y):
        """
        Returns the value of the square at position (x, y)
        """
        return int(self.board[y][x])
    
    def get_legal_actions(self):
        """
        Returns a list of legal actions
        """
        return self.actions
    
    def get_current_state(self):
        """
        Returns the current board state.
        """
        return self.board
    
    def get_last_state(self):
        """
        Returns the board state we were in before this one, or None of it does not exist.
        """
        if len(self.history) == 0:
            return None
        else:
            return self.history[-1]
        
    def get_last_move_hist(self):
        """
        Returns the distances and direction that tiles in the previous board state moved 
        in order to reach the current state
        """
        if len(self.move_history) == 0:
            return None
        else:
            return self.move_history[-1]
    
    def get_last_fill_coord(self):
        """
        Returns the last coordinate filled by a random number
        """
        if len(self.fill_history) == 0:
            return None
        else:
            return self.fill_history[-1]
        
    def get_last_merge_hist(self):
        """
        Returns an array with 1's indicating where the previous state merged tiles while
        moving to the new state. 0 everywhere else.
        """
        if len(self.merge_history) == 0:
            return None
        else:
            return self.merge_history[-1]
        
    def compute_dest_array(self, prev_state, current_state, move_hist):
        """
        Considers the previous state, the current state, and the move history to compute
        where map all locations in the previous state to locations in the current state
        
        Inputs:
        ------
        prev_state: an array of the previous board state
        current_state: an array of the current board state
        move_hist: a tuple of (move_dist, movement) where move_dist is an array 
                    listing the distance travelled by each tile and movement
                    indicates the direction travelled.
                    (directions mapped as "left" = 0, "up" = 1, "right" = 2, "down" = 3)
                    
        Outputs:
        -------
        dest_array: a destination array mapping each tile in prev_state to its destination 
                    in current_state
                    (uses inverse notation (y, x))
        """
        move_dist, movement = move_hist
        rows, columns = prev_state.shape
        # data type is a list of 64-byte integers [y, x]
        dest_array = np.zeros((rows, columns), dtype='2int64')
        
        for row in range(0, rows):
            for col in range(0, columns):
                shift_x = 0
                shift_y = 0
                
                # left = sub dist in X
                if movement == 0:
                    shift_x = -move_dist[row][col]
                    
                # up = add dist in Y
                elif movement == 1:
                    shift_y = move_dist[row][col]
                    
                # right = add dist in X
                elif movement == 2:
                    shift_x = move_dist[row][col]
                    
                # down = sub dist in Y
                elif movement == 3:
                    shift_y = -move_dist[row][col]
                else:
                    print('Error. Please pass in the correct movement.')
                
                dest_array[row][col] = [row+shift_y, col+shift_x]
                
        return dest_array
    
    def get_dest_array(self):
        """
        Considers the previous state, the current state, and the move history to compute
        where map all locations in the previous state to locations in the current state.
        dest_array uses inverse notation (y, x)
        """
        prev_state = self.get_last_state()
        current_state = self.get_current_state()
        move_hist = self.get_last_move_hist()
        return self.compute_dest_array(prev_state, current_state, move_hist)
        
        
    def undo_last_move(self):
        """
        Reverts the board state to the next most recent state in our history, if it exists
        """
        if len(self.history) != 0:
            last_state = self.history.pop()
            last_move_hist = self.move_history.pop()
            self.board = last_state 
            

In [24]:
game = Game_2048(m = 4, n = 4, prime = 2, goal = 32)
x = np.arange(4).reshape((2,2))

In [39]:
print("Starting game now.")
while True:
    #plt.matshow(game.board)
    #plt.show()
    print(game.board)
    if game.is_game_over() == 1:
        if game.is_win() == 0:
            print("Sorry, you lost.")
        else:
            print("You won!")
        break
    action = input()
    if action.lower() == "exit":
        print("Ending game now.")
        break
    game.move(action)

Starting game now.
[[0. 0. 0. 0.]
 [0. 2. 0. 0.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]]
left
[[0. 2. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]]
left
[[2. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 2. 0.]
 [0. 0. 0. 0.]]
left
[[2. 0. 0. 0.]
 [2. 2. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 0.]]
exit
Ending game now.


In [83]:
class StartGame:
    BG = "#fbf8ef"
    TITLE = "2048 Supreme"
    TITLE_FONT = ("Helvetica", 28)
    TILE_FONT = ("Helvetica", 16)
    TILE_SIZE = 100
    TILE_SPACING = 15
    TILE_MOVEMENT_SPEED = 1
    TILE_OUTLINE = "#b9ab9e"
    TILE_FILL = {0: "#c9bdb1", 2: "#eee4da", 4: "#ece0c8", 8: "#f2b179", 
                   16: "#f29762", 32: "#f57c5f", 64: "#f65d3b" , 128: "#edce71", 
                   256: "#f9d067", 512: "#e4c02a", 1024: "#e4b714", 
                   2048: "#fac42e"}
    TILE_TEXT_COLOR = {0: None, 2: "#796860", 4: "#796860", 8: "white", 16: "white",
                   32: "white", 64: "white", 128: "white", 256: "white", 
                   512: "white", 1024: "white", 2048: "white"}
    TILE_REPO = {}
    
    def __init__(self):
        self.GAME = Game_2048()
        self.numRows = self.GAME.m
        self.numCols = self.GAME.n
        self.boardHeight = self.numRows * self.TILE_SIZE + (self.numRows+1) * self.TILE_SPACING
        self.boardWidth = self.numCols * self.TILE_SIZE + (self.numCols+1) * self.TILE_SPACING
        self.TILE_REPO = {}
        
        self.root = tk.Tk()
        self.root.title(self.TITLE)
        self.root.configure(bg=self.BG, width=self.boardWidth * 2, height=self.boardHeight*2)
        
        title=tk.Label(self.root, text=self.TITLE, bg=self.BG, fg="#756e64", font=self.TITLE_FONT)
        title.pack()
        
        
        # Main Board
        boardFrame = tk.Frame(self.root, width=self.boardWidth+20, height=self.boardHeight+20, bg=self.BG, padx=10, pady=10)
        boardFrame.pack()
        
        self.canvas = tk.Canvas(boardFrame, width=self.boardWidth, height=self.boardHeight, bg="#baaea0", borderwidth=0)
        self.canvas.pack()
     
        self.init_board()
        
        # Command Frame and Buttons
        commandFrame=tk.Frame(self.root, width=self.boardWidth, height=100, bg=self.BG, pady=10, padx=10)
        commandFrame.pack()
        
        self.set_command_buttons(commandFrame)
        
        # Return Button
        returnButton = tk.Button(self.root, text="Quit", padx=5, pady=5)
        returnButton.pack(side=tk.BOTTOM)
        returnButton.bind("<Button 1>", self.quit)
        
        self.root.bind_all("<Escape>", self.quit)
        
        self.root.mainloop()
        
    def create_frames(self, root):
        pass
    
    def create_widgets(self):
        pass
    
    def set_command_buttons(self, frame):
        upButton=tk.Button(frame, text="Up", padx=5, pady=5)
        leftButton=tk.Button(frame, text="Left", padx=5, pady=5)
        rightButton=tk.Button(frame, text="Right", padx=5, pady=5)
        downButton=tk.Button(frame, text="Down", padx=5, pady=5)
        
        upButton.bind("<Button 1>", self.move)
        leftButton.bind("<Button 1>", self.move)
        rightButton.bind("<Button 1>", self.move)
        downButton.bind("<Button 1>", self.move)
        
        upButton.pack(side=tk.TOP)
        leftButton.pack(side=tk.LEFT)
        rightButton.pack(side=tk.RIGHT)
        downButton.pack(side=tk.BOTTOM)
        
        self.root.bind_all("<w>", self.move)
        self.root.bind_all("<a>", self.move)
        self.root.bind_all("<s>", self.move)
        self.root.bind_all("<d>", self.move)
        
        self.root.bind_all("<Up>", self.move)
        self.root.bind_all("<Left>", self.move)
        self.root.bind_all("<Down>", self.move)
        self.root.bind_all("<Right>", self.move)
        
    def move(self, event):
        if event.char == "??":
            command = str(event.widget.cget("text")).lower().strip()
        else:
            command = str(event.keysym).lower().strip()
        if command in self.GAME.get_legal_actions():
            self.GAME.move(command)
            #print("\ndest_arr is: \n{0}\nfill_coord is: {1}\n".format(dest_arr, fill_coord))

            self.animate_move()
        
    def quit(self, event):
        self.root.destroy()
        
    def find_tile_placement(self, col, row):
        """
        Finds the pixel placement of a tile in a given (col, row) or (x, y) location, account for
        TILE_SIZE and TILE_SPACING.
        """
        start_x = self.TILE_SIZE*col + self.TILE_SPACING*(col+1)
        start_y = self.TILE_SIZE*row + self.TILE_SPACING*(row+1)
        end_x = self.TILE_SIZE*(col+1) + self.TILE_SPACING*(col+1)
        end_y = self.TILE_SIZE*(row+1) + self.TILE_SPACING*(row+1)
        
        return start_x, start_y, end_x, end_y
    
    def find_text_placement(self, col, row):
        """
        Finds the pixel placement of the text of a tile in a given (col, row) = (x, y) location,
        accounting for TILE_SIZE and TILE_SPACING. Text is CENTERED at center of tile.
        
        Returns (center_x, center_y) where this is the location where text should be centered at.
        """
        center_x = self.TILE_SIZE*(col+0.5) + self.TILE_SPACING*(col+1)
        center_y = self.TILE_SIZE*(row+0.5) + self.TILE_SPACING*(row+1)
        return center_x, center_y
    
    def create_tile_tk(self, col, row, num):
        """
        Creates a tk shape object at specified (col, row) = (x, y) location in the board with
        the given num. Also creates a tk text object for the given num on the board.
        
        Returns a tuple (tk_shape, tk_text)
        """
        num_bg, num_text_color = self.get_tile_bg_and_text_color(num)
        start_x, start_y, end_x, end_y = self.find_tile_placement(col, row)
        tile = self.canvas.create_rectangle(start_x, start_y, end_x, end_y,
                                                 outline=self.TILE_OUTLINE, fill=num_bg)
    
        if num != 0:
            text_center_x, text_center_y = self.find_text_placement(col, row)
            tile_text = self.canvas.create_text(text_center_x, text_center_y, 
                                                text=str(num), font=self.TILE_FONT, fill=num_text_color)
        else:
            tile_text = None
            
        return (tile, tile_text)
    
    def add_tile_to_repo(self, tile, tile_text, col, row):
        """
        Adds a given tk shape object (tile) and tk text object (tile) and adds it to TILE_REPO as
        a tuple with key (row, col) = (y, x)
        """
        self.TILE_REPO[(row, col)] = (tile, tile_text)
        
    def pop_tile_from_repo(self, col, row):
        """
        Removes the tk shape object mapped to key (row, col) in TILE_REPO and returns it or None 
        if it did not exist
        """
        tile, tile_text = self.TILE_REPO.pop((row, col), None)
        return (tile, tile_text)
        
    def init_board(self):
        """
        Initializes the empty board. Sets up a ZERO tile below every tile. Then finds the initial
        numbers, creates tiles for them, and places them on the board. Also maps the tile objects
        to their (x, y) location on the board in our TILE_REPO object
        """
        # First initialize all tiles to ZERO. These will NEVER be moved.
        for col in range(self.numCols):
            for row in range(self.numRows):
                zero_tile, tile_text = self.create_tile_tk(col, row, 0)
                
        # Now scan array to find all NONZERO tiles, create them, and store them in TILE_REPO. 
        # These WILL be moved at some point in the future.
        for col in range(self.numCols):
            for row in range(self.numRows):
                num = self.GAME.get_val(x=col, y=row)
                if num != 0:
                    init_tile, tile_text = self.create_tile_tk(col, row, num)
                    self.add_tile_to_repo(init_tile, tile_text, col, row)
    
    
    # sort of a useless function now
    def update_board(self):
        for col in range(self.numCols):
            for row in range(self.numRows):
                num = self.GAME.get_val(x=col, y=row)
                num_bg, num_text_color = self.get_tile_bg_and_text_color(num)
                
                start_x = self.TILE_SIZE*col + self.TILE_SPACING*(col+1)
                start_y = self.TILE_SIZE*row + self.TILE_SPACING*(row+1)
                end_x = self.TILE_SIZE*(col+1) + self.TILE_SPACING*(col+1)
                end_y = self.TILE_SIZE*(row+1) + self.TILE_SPACING*(row+1)
                init_tile = self.canvas.create_rectangle(start_x, start_y, end_x, end_y, 
                                             outline=self.TILE_OUTLINE, fill=num_bg)
                if num != 0:
                    text_center_x = self.TILE_SIZE*(col+0.5) + self.TILE_SPACING*(col+1)
                    text_center_y = self.TILE_SIZE*(row+0.5) + self.TILE_SPACING*(row+1)
                    self.canvas.create_text(text_center_x, text_center_y, 
                                            text=str(num), font=self.TILE_FONT, fill=num_text_color)
   

    def animate_move(self):
        """
        Animates a movement
        """
        prev_state = self.GAME.get_last_state()
        dest_arr = self.GAME.get_dest_array()
        fill_coord = self.GAME.get_last_fill_coord()
        merge_arr = self.GAME.get_last_merge_hist()
        
        numRows, numCols = prev_state.shape
        
        # sets up our tiles to be used for animation
        tiles = []
        # dictionary mapping a (x, y) coord to [list of (tile merging there, 1 if arrived 0 else)]
        merges = {}
        # TODO: What do we do with TILE_REPO_COPY?
        TILE_REPO_COPY = self.TILE_REPO.copy()
        print("TILE_REPO: {0}\n".format(self.TILE_REPO))
        for coord, tile_and_text in TILE_REPO_COPY.items():
            # Tile repo uses inverse (y, x) notation
            row, col = coord
            num = self.GAME.get_val(x=col, y=row)
            num_bg, num_text_color = self.get_tile_bg_and_text_color(num)
            dest_y, dest_x = dest_arr[row][col]
            print("start_x: {0}, start_y: {1}, dest_x: {2}, dest_y: {3}".format(col, row, dest_x, dest_y))
            tk_tile, tk_tile_text = self.pop_tile_from_repo(col, row)
            
            """
            # creates animation tile
            # DEFINES each tile by the top left corner of the SQUARE it corresponds to in PIXELS
            start_x_pixels, start_y_pixels, _useless1_, _useless2_ = self.find_tile_placement(col, row)
            dest_x_pixels, dest_y_pixels, _useless3_, _useless4_ = self.find_tile_placement(dest_x, dest_y)
            animation_tile = self.Tile(start_x=start_x_pixels, start_y=start_y_pixels, 
                                       dest_x=dest_x_pixels, dest_y=dest_y_pixels, 
                                       background=num_bg, num=num, text_color=num_text_color,
                                       tk_tile = tk_tile, tk_tile_text = tk_tile_text)
            """
            #original attempt tries to define movement speed in tiles per second
            animation_tile = self.Tile(start_x=col, start_y=row, 
                                       dest_x=dest_x, dest_y=dest_y, 
                                       background=num_bg, num=num, text_color=num_text_color,
                                       tk_tile = tk_tile, tk_tile_text = tk_tile_text)
            tiles.append(animation_tile)
            
            # 1 if a merge occurs at destination, 0 otherwise
            merge = merge_arr[row][col]
            if merge == 1:
                # add list of tiles merging at destination!
                # merges will use inverse notation (y, x)
                if (dest_y, dest_x) in merges:
                    merge_list = merges[(dest_y, dest_x)]
                    merge_list.append(animation_tile)
                else:
                    # inits counts of tiles arrived as 0
                    merge_list = [0, animation_tile]
                    merges[(dest_y, dest_x)] = merge_list
            
        # TODO: HANDLE TILE MERGES
        # DICT mapping \{ merge destination: [count_tiles_arrived, *tiles*]\}
        while len(tiles) != 0:
            print('Preparing to begin animation step.\nTiles are: {0}\n'.format(tiles))
            tiles, merges = self.animate_step(tiles, merges)
            time.sleep(.5)
        
            
        return True
        # Once we have moved a tile, we need to act it back to our TILE_REPO mapped to the new coords
        # (The old coord:tile) map was deleted by pop() already so we gucci there
    
    def animate_step(self, tiles, merge_dict):
        """
        Runs a single step of the animation using the given [tiles] list and merge_dict.
        
        Returns the new [tiles] list after the animation step. Removes tiles from [tiles] 
        if they have reached their destination. 
        
        merge_dict maps a dest coord (x, y) to a merge_list of [count_tiles_arrived, *tiles merging at coord*].
        If the destination is a merge and we have arrived, then increment count_tiles_arrived by 1. If setting 
        this value tells us all merging tiles have arrived (if = len(merge_list)-1), then we proceed with merge.
        So we destroy all the merging tiles and replace with a new tile.
        """      
        # animation step
        new_tiles = []
        print('Beginning animation step.\nTiles are: {0}\nMerge_Dict is: {1}\n'.format(tiles, merge_dict))
        for move_tile in tiles: 
            # retrieves movement distance in terms of squares
            print('Preparing to move move_tile defined by: {0}\n'.format(move_tile))
            # currently movement speed defined in TILES per movement - not PIXELS, which you want
            x_move, y_move = move_tile.move(self.TILE_MOVEMENT_SPEED)
            print('In terms of squares, x_move: {0}, y_move: {1}\n'.format(x_move, y_move))

            # convert movement to pixels, accounting for tile spacing
            # NOTE: both the tk_tile and tk_tile_text will move the SAME distance
            x_move = self.TILE_SIZE*x_move + self.TILE_SPACING*(x_move)
            y_move = self.TILE_SIZE*y_move + self.TILE_SPACING*(y_move)     
            print('In terms of pixels, x_move: {0}, y_move: {1}\n'.format(x_move, y_move))

            tk_tile, tk_tile_text = move_tile.get_tk_obj()
            print('Trying to move tk_tile: {0}\ntk_tile_text: {1}\n'.format(tk_tile, tk_tile_text))
            
            # TODO: CURRENTLY NOTHING IS MOVING ON THE SCREEN?
            self.canvas.move(tk_tile, x_move, y_move)
            self.canvas.move(tk_tile_text, x_move, y_move)

            # if tile not yet at destination, then add back to cycle
            if move_tile.at_dest():
                # TODO: if all tiles have arrived
                # destroy them and replace with a new tile
                dest = move_tile.get_dest()
                
                # our move_tile is part of a merging operation
                # merge_dict uses inverse notation (y, x)
                if dest in merge_dict:
                    # merge_list is a list of [count_tiles_arrived, *tiles*]
                    merge_list = merge_dict[dest]
                    # increment the count of tiles arriving by 1
                    merge_list[0] += 1
                    # indicates all tiles have arrived
                    if merge_list[0] == (len(merge_list) - 1):
                        # delete old tk shapes
                        for merge_tile in merge_list[1:]:
                            tk_tile, tk_tile_text = merge_tile.get_tk_obj()
                            self.canvas.delete(tk_tile)
                            self.canvas.delete(tk_tile_text)
                        # create new tk shape and add to TILE_REPO  
                        dest_y, dest_x = dest
                        num = self.GAME.get_val(x=dest_x, y=dest_y)
                        new_merged_tile, new_merged_tile_text = self.create_tile_tk(col=dest_x, row=dest_y, num=num)
                        self.add_tile_to_repo(tile=new_merged_tile, tile_text=new_merged_tile_text, 
                                              col=dest_x, row=dest_y)
                        # now that the merge is complete, remove the list from the dict
                        merge_dict.pop(dest, None)                                                       
            # if not at dest then add back to our queue        
            else:
                new_tiles.append(move_tile)
            
            # let's say we have the existing tiles from the old board
            # then we move the tile if it needs to be moved(all move to same direction)
            # maintain a ZERO tile underneath every tile that will never move to preserve board space?
            # then we only have COLORED tiles to move. so we animate all their movement
            # if we notice that a MERGE occurs -> then when the two tiles reach their DEST, we destroy them 
            # and replace the DEST with the new tile
        return new_tiles, merge_dict
                
                
    def get_tile_bg_and_text_color(self, num):
        """
        Takes in a number and returns the corresponding background and text_color
        """
        if num in self.TILE_FILL:
            num_bg = self.TILE_FILL[num]
            num_text_color = self.TILE_TEXT_COLOR[num]
        else:
            num_bg = self.TILE_FILL[2048]
            num_text_color = self.TILE_TEXT_COLOR[2048]
            
        return (num_bg, num_text_color)
        
        
    class Tile:
        """
        Objects to be used for animation of tiles
        """
        def __init__(self, start_x, start_y, dest_x, dest_y, background, num, text_color, tk_tile, tk_tile_text):
            self.current_x = start_x
            self.current_y = start_y
            self.dest_x = dest_x
            self.dest_y = dest_y
            self.bg = background
            self.num = num
            self.text_color = text_color
            self.tk_tile = tk_tile
            self.tk_tile_text = tk_tile_text
            
        def get_tk_obj(self):
            """
            Returns a tuple of our tk objects (tk_tile, tk_tile_text)
            """
            return (self.tk_tile, self.tk_tile_text)
        
        def get_dest(self):
            """
            Returns the destination of our tile as a tuple (dest_x, dest_y)
            """
            return (self.dest_x, self.dest_y)
        
        def move(self, movement_speed):
            """
            Moves the object from our (current_x, current_y) -> (dest_x, dest_y) in a straight line.
            
            Updates our current position and returns the amount we moved. If we would overshoot then 
            moves only as far as necessary.
            """
            
            x_dist = self.dest_x - self.current_x
            y_dist = self.dest_y - self.current_y
            
            x_dir = np.sign(x_dist)
            y_dir = np.sign(y_dist)

            # if we would overshoot the distance with the movement, just take the distance as the movement
            # factors: if x_dist is ZERO, or x_dist < movement speed, then we only move just enough
            # x movement determined by sign of x_dist
            x_movement = x_dir * min(abs(x_dist), movement_speed)
            y_movement = y_dir * min(abs(y_dist), movement_speed)
            
            new_x = self.current_x + x_movement
            new_y = self.current_y + y_movement
            
            self.current_x = new_x
            self.current_y = new_y
            
            return (x_movement, y_movement)
            
        def at_dest(self):
            """
            Returns true if we've reached our goal. False otherwise.
            """
            if self.current_x == self.dest_x and self.current_y == self.dest_y:
                return True
            else:
                return False
            
        def __str__(self):
            return "current_x: {0}, current_y: {1}, dest_x: {2}, dest_y: {3}".format(self.current_x, self.current_y, 
                                                                                      self.dest_x, self.dest_y)
        
        def __eq__(self, other): 
            try:
                return self.tk_tile == other.tk_tile
            except AttributeError:
                print('Exception occured')
                return False
        

In [84]:
game = StartGame()
print(game.TILE_REPO)

TILE_REPO: {(1, 0): (17, 18), (3, 0): (19, 20)}

start_x: 0, start_y: 1, dest_x: 3, dest_y: 1
start_x: 0, start_y: 3, dest_x: 3, dest_y: 3
Preparing to begin animation step.
Tiles are: [<__main__.StartGame.Tile object at 0x000001E96D6E96D8>, <__main__.StartGame.Tile object at 0x000001E96D6E9B00>]

Beginning animation step.
Tiles are: [<__main__.StartGame.Tile object at 0x000001E96D6E96D8>, <__main__.StartGame.Tile object at 0x000001E96D6E9B00>]
Merge_Dict is: {}

Preparing to move move_tile defined by: current_x: 0, current_y: 1, dest_x: 3, dest_y: 1

In terms of squares, x_move: 1, y_move: 0

In terms of pixels, x_move: 115, y_move: 0

Trying to move tk_tile: 17
tk_tile_text: 18

Preparing to move move_tile defined by: current_x: 0, current_y: 3, dest_x: 3, dest_y: 3

In terms of squares, x_move: 1, y_move: 0

In terms of pixels, x_move: 115, y_move: 0

Trying to move tk_tile: 19
tk_tile_text: 20

Preparing to begin animation step.
Tiles are: [<__main__.StartGame.Tile object at 0x0000

In [92]:
array = np.zeros((3, 3))
array[1, 2] = 1
print(array)

[[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]


In [15]:
array = np.array([(1,2), (2,3)])
print(array)

[[1 2]
 [2 3]]


In [129]:
tile = StartGame.Tile(0, 0, 5, 5)
print(str(tile))
tile.move(10)
print(str(tile))
print(tile.is_goal())

current_x: 0, current_y: 0, dest_x: 5, dest_y: 5

current_x: 5, current_y: 5, dest_x: 5, dest_y: 5

True


In [7]:
dicto = {'hello': 'me'}
r = dicto.pop('hello', None)
print(r)
print(dicto)

me
{}
